In [21]:

import numpy as np
import cv2
import pandas as pd
from scipy.stats import skew, kurtosis, entropy
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from skimage.measure import label, regionprops
from scipy.ndimage import convolve
from math import sqrt
import os
from tqdm import tqdm
import mahotas


In [5]:
def lbp_feature(image):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    i_max = np.max(image)
    i_min = np.min(image)
    if i_max - i_min != 0:
        image = (image - i_min) / (i_max - i_min)
    lbp = local_binary_pattern(image, R=1, P=8, method="uniform")  
    hist, bins = np.histogram(lbp.ravel(), bins=10, range=(0, 10))
    return {f"LBP_{i}": hist[i] for i in range(len(hist))}

In [6]:
def texture_feature(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray_img],[0],None, [256], [0,256])
    hist = hist/hist.sum()

    intensities = np.arange(256)
    mean = np.sum(intensities * hist.flatten())
    std = np.sqrt(np.sum((intensities-mean)**2 *hist.flatten()))
    uniformity = np.sum(hist.flatten()**2)
    third_moment = np.sum((intensities-mean)**3 * hist.flatten())
    return {
        "texture_mean": mean,
        "texture_std": std,
        "texture_uniformity": uniformity,
        "texture_third_moment": third_moment
    }

In [7]:
def color_feature(image):
    ##rgb
    b, g, r = cv2.split(image)
    mean_r, mean_g, mean_b = np.mean(r), np.mean(g), np.mean(b)
    rs, gs, bs = sqrt(mean_r), np.sqrt(mean_g), np.sqrt(mean_b)
    return {"mean_r": mean_r,
        "mean_g": mean_g,
        "mean_b": mean_b,
        "red_sqr": rs,
        "green_sqr": gs,
        "blue_sqr": bs,
    }

In [ ]:
def enhanced_color_feature(image):

    b, g, r = cv2.split(image)
    mean_r, mean_g, mean_b = np.mean(r), np.mean(g), np.mean(b)
    std_r, std_g, std_b = np.std(r), np.std(g), np.std(b)
    rs, gs, bs = np.sqrt(mean_r), np.sqrt(mean_g), np.sqrt(mean_b) 
    skew_r, skew_g, skew_b = skew(r.ravel()), skew(g.ravel()), skew(b.ravel())
    kurtosis_r, kurtosis_g, kurtosis_b = kurtosis(r.ravel()), kurtosis(g.ravel()), kurtosis(b.ravel())

    ## HSV 
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)
    mean_h, mean_s, mean_v = np.mean(h), np.mean(s), np.mean(v)
    std_h, std_s, std_v = np.std(h), np.std(s), np.std(v)
    hs, ss, vs = np.sqrt(mean_h), np.sqrt(mean_s), np.sqrt(mean_v)
    skew_h, skew_s, skew_v = skew(h.ravel()), skew(s.ravel()), skew(v.ravel())
    kurtosis_h, kurtosis_s, kurtosis_v = kurtosis(h.ravel()), kurtosis(s.ravel()), kurtosis(v.ravel())
    
    ## Ycrbr
    ycrcb_img = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb_img)
    mean_y, mean_cr, mean_cb = np.mean(y), np.mean(cr), np.mean(cb)
    std_y, std_cr, std_cb = np.std(y), np.std(cr), np.std(cb)
    ys, crs, cbs = np.sqrt(mean_y), np.sqrt(mean_cr), np.sqrt(mean_cb)
    skew_y, skew_cr, skew_cb = skew(y.ravel()), skew(cr.ravel()), skew(cb.ravel())
    kurtosis_y, kurtosis_cr, kurtosis_cb = kurtosis(y.ravel()), kurtosis(cr.ravel()), kurtosis(cb.ravel())

    ## Lab
    lab_img = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    l, a, b_lab = cv2.split(lab_img)
    mean_l, mean_a, mean_b_lab = np.mean(l), np.mean(a), np.mean(b_lab)
    std_l, std_a, std_b_lab = np.std(l), np.std(a), np.std(b_lab)
    ls, as_, bs_lab = np.sqrt(mean_l), np.sqrt(mean_a), np.sqrt(mean_b_lab)
    skew_l, skew_a, skew_b_lab = skew(l.ravel()), skew(a.ravel()), skew(b_lab.ravel())
    kurtosis_l, kurtosis_a, kurtosis_b_lab = kurtosis(l.ravel()), kurtosis(a.ravel()), kurtosis(b_lab.ravel())
    
    return {
        "mean_r": mean_r, "mean_g": mean_g, "mean_b": mean_b,
        "std_r": std_r, "std_g": std_g, "std_b": std_b,
        "red_sqr": rs, "green_sqr": gs, "blue_sqr": bs,
        "skew_r": skew_r, "skew_g": skew_g, "skew_b": skew_b,
        "kurtosis_r": kurtosis_r, "kurtosis_g": kurtosis_g, "kurtosis_b": kurtosis_b,

        "mean_h": mean_h, "mean_s": mean_s, "mean_v": mean_v,
        "std_h": std_h, "std_s": std_s, "std_v": std_v,
        "hue_sqr": hs, "sat_sqr": ss, "val_sqr": vs,
        "skew_h": skew_h, "skew_s": skew_s, "skew_v": skew_v,
        "kurtosis_h": kurtosis_h, "kurtosis_s": kurtosis_s, "kurtosis_v": kurtosis_v,
        
        "mean_y": mean_y, "mean_cr": mean_cr, "mean_cb": mean_cb,
        "std_y": std_y, "std_cr": std_cr, "std_cb": std_cb,
        "y_sqr": ys, "cr_sqr": crs, "cb_sqr": cbs,
        "skew_y": skew_y, "skew_cr": skew_cr, "skew_cb": skew_cb,
        "kurtosis_y": kurtosis_y, "kurtosis_cr": kurtosis_cr, "kurtosis_cb": kurtosis_cb,

        "mean_l": mean_l, "mean_a": mean_a, "mean_b_lab": mean_b_lab,
        "std_l": std_l, "std_a": std_a, "std_b_lab": std_b_lab,
        "l_sqr": ls, "a_sqr": as_, "b_lab_sqr": bs_lab,
        "skew_l": skew_l, "skew_a": skew_a, "skew_b_lab": skew_b_lab,
        "kurtosis_l": kurtosis_l, "kurtosis_a": kurtosis_a, "kurtosis_b_lab": kurtosis_b_lab
    }


In [9]:
def zernike_feature(image, degree=8):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    
    mask = np.zeros(new_img.shape, dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)  
    
    # Convert mask to binary for mahotas
    binary_mask = (mask > 0).astype(np.uint8)
    

    (x_center, y_center), radius = cv2.minEnclosingCircle(contour)

    radius = int(np.ceil(radius))
    

    x_center = int(x_center)
    y_center = int(y_center)
    x1 = max(x_center - radius, 0)
    y1 = max(y_center - radius, 0)
    x2 = x_center + radius
    y2 = y_center + radius
    
    cropped_mask = binary_mask[y1:y2, x1:x2]
    
    # padding
    h, w = cropped_mask.shape
    if h != w:
        size = max(h, w)
        square_mask = np.zeros((size, size), dtype=np.uint8)
        y_offset = (size - h) // 2
        x_offset = (size - w) // 2
        square_mask[y_offset:y_offset+h, x_offset:x_offset+w] = cropped_mask
    else:
        square_mask = cropped_mask

    effective_radius = square_mask.shape[0] // 2
    zernike_moments = mahotas.features.zernike_moments(square_mask, effective_radius, degree)
    
    return {f'zernike_{i}': zernike_moments[i] for i in range(len(zernike_moments))}

In [10]:
def color_structure_descriptor(image, grid_size=(8, 8), bins=32):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    
    height, width = hsv_image.shape[:2]
    cell_h, cell_w = height // grid_size[0], width // grid_size[1]

    
    hist_bins = [bins] 

    
    csd = []

    for row in range(grid_size[0]):
        for col in range(grid_size[1]):
            x_start, y_start = col * cell_w, row * cell_h
            x_end, y_end = x_start + cell_w, y_start + cell_h
            cell = hsv_image[y_start:y_end, x_start:x_end]

            hist = cv2.calcHist([cell], [0], None, hist_bins, [0, 180])
            hist = hist.flatten() 
            hist = hist / hist.sum() if hist.sum() != 0 else hist

            csd.extend(hist)

    return {f"csd_{i}":csd[i] for i in range(len(csd))}


In [11]:
def hu_moments(image):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    moments = cv2.moments(contour)
    hu_moments = cv2.HuMoments(moments)
    hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)
    return {f"hu_{i}": hu_moments[i] for i in range(len(hu_moments))}

In [12]:
def basic_feature(image):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # new_img = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    #                            cv2.THRESH_BINARY_INV, 11, 2)
    area = np.count_nonzero(new_img)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(contour)
    
    length = x + w
    width = y + h
    ratio = length / width
    
    ellipse = cv2.fitEllipse(contour)
    
    major_axis_length = max(ellipse[1])
    minor_axis_length = min(ellipse[1])

    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    return {
        "area": area,
        "length": length,
        "width": width,
        "ratio": ratio,
        "major_axis_length": major_axis_length,
        "minor_axis_length": minor_axis_length,
        "convex_hull_area": hull_area,
        "convex_hull_perimeter": hull_perimeter,
    }

In [13]:
from scipy.ndimage import convolve
def extract_edge_histogram_features(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    kernels = {
        "vertical": np.array([[-1,  2, -1],
                              [-1,  2, -1],
                              [-1,  2, -1]], dtype=np.float32),
        "horizontal": np.array([[-1, -1, -1],
                                [ 2,  2,  2],
                                [-1, -1, -1]], dtype=np.float32),
        "diag_45": np.array([[-1, -1,  2],
                             [-1,  2, -1],
                             [ 2, -1, -1]], dtype=np.float32),
        "diag_135": np.array([[ 2, -1, -1],
                              [-1,  2, -1],
                              [-1, -1,  2]], dtype=np.float32),
        "non_directional": np.array([[1,  1,  1],
                                     [1, -8,  1],
                                     [1,  1,  1]], dtype=np.float32)
    }
    
    features = {}
    total_energy = 0.0
    
    # Convolve the image with each kernel and compute the energy.
    for key, kernel in kernels.items():
        response = convolve(gray.astype(np.float32), kernel, mode="reflect")
        energy = np.sum(np.abs(response))
        features[f"edge_energy_{key}"] = energy
        total_energy += energy
    
    if total_energy > 0:
        for key in list(features.keys()):
            features[key] /= total_energy

    return features

In [ ]:
# def extract_edge_histogram_features(image):
#     # Convert to grayscale if necessary
#     if len(image.shape) > 2:
#         gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     else:
#         gray = image.copy()
    
#     # Define convolution kernels for a variety of edge types.
#     kernels = {
#         "vertical": np.array([[-1,  2, -1],
#                               [-1,  2, -1],
#                               [-1,  2, -1]], dtype=np.float32),
#         "horizontal": np.array([[-1, -1, -1],
#                                 [ 2,  2,  2],
#                                 [-1, -1, -1]], dtype=np.float32),
#         "diag_45": np.array([[-1, -1,  2],
#                              [-1,  2, -1],
#                              [ 2, -1, -1]], dtype=np.float32),
#         "diag_135": np.array([[ 2, -1, -1],
#                               [-1,  2, -1],
#                               [-1, -1,  2]], dtype=np.float32),
#         "non_directional": np.array([[1,  1,  1],
#                                      [1, -8,  1],
#                                      [1,  1,  1]], dtype=np.float32),
#         # Sobel Operators
#         "sobel_x": np.array([[-1, 0, 1],
#                              [-2, 0, 2],
#                              [-1, 0, 1]], dtype=np.float32),
#         "sobel_y": np.array([[-1, -2, -1],
#                              [ 0,  0,  0],
#                              [ 1,  2,  1]], dtype=np.float32),
#         # Prewitt Operators
#         "prewitt_x": np.array([[-1, 0, 1],
#                                [-1, 0, 1],
#                                [-1, 0, 1]], dtype=np.float32),
#         "prewitt_y": np.array([[-1, -1, -1],
#                                [ 0,  0,  0],
#                                [ 1,  1,  1]], dtype=np.float32),
#         # Laplacian Operators
#         "laplacian_4": np.array([[0,  1,  0],
#                                  [1, -4,  1],
#                                  [0,  1,  0]], dtype=np.float32),
#         "laplacian_8": np.array([[1,  1,  1],
#                                  [1, -8,  1],
#                                  [1,  1,  1]], dtype=np.float32),
#         # Roberts Cross Operators
#         "roberts_x": np.array([[1, 0],
#                                [0, -1]], dtype=np.float32),
#         "roberts_y": np.array([[0, 1],
#                                [-1, 0]], dtype=np.float32),
#         # Scharr Operators
#         "scharr_x": np.array([[-3,  0,  3],
#                               [-10, 0, 10],
#                               [-3,  0,  3]], dtype=np.float32),
#         "scharr_y": np.array([[-3, -10, -3],
#                               [ 0,   0,  0],
#                               [ 3,  10,  3]], dtype=np.float32),
#     }
    
#     features = {}
#     total_energy = 0.0
    
#     # Convolve the image with each kernel and compute the energy.
#     for key, kernel in kernels.items():
#         response = convolve(gray.astype(np.float32), kernel, mode="reflect")
#         energy = np.sum(np.abs(response))
#         features[f"edge_energy_{key}"] = energy
#         total_energy += energy
    
#     # Normalize the features by the total energy.
#     if total_energy > 0:
#         for key in features.keys():
#             features[key] /= total_energy

#     return features

In [15]:
def compute_glcm_descriptor(image):
    if image is None:
        return None
    
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    distance = [3]  
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4] 
    properties = ['contrast', 'correlation', 'energy', 'homogeneity']  
    
    glcm = graycomatrix(image, distances=distance, angles=angles, symmetric=True, normed=True)
    
    features = []
    for prop in properties:
        feature = graycoprops(glcm, prop).flatten()
        features.extend(feature)
    
    return np.array(features)

In [16]:
def create_gabor_filter(size, u0, v0, delta_x, delta_y):
    #size: kernel size
    #u0, v0: spatial frequency points
    #delta_x, delta_y: spatial scales

    y, x = np.mgrid[-size//2:size//2, -size//2:size//2]
    
    gaussian = np.exp(-0.5 * (x**2/delta_x**2 + y**2/delta_y**2))

    sinusoid = np.exp(-2j * np.pi * (u0*x + v0*y))
    
    return gaussian * sinusoid

def compute_gist_descriptor(image):

    if image is None:
        return None
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.astype(np.float32) / 255.0
    
    local_mean = cv2.GaussianBlur(image, (5, 5), 1.0)
    variance = cv2.GaussianBlur(image**2, (5, 5), 1.0) - local_mean**2
    variance[variance < 0] = 0 
    local_std = np.sqrt(variance)
    image = (image - local_mean) / (local_std + 1e-8)
    
    features = []
    scales = [2, 4, 8, 16]
    orientations = 8
    
    for delta in scales:
        for theta in range(orientations):
            angle = theta * np.pi / orientations
            u0 = np.cos(angle) / delta
            v0 = np.sin(angle) / delta
            
            gabor_filter = create_gabor_filter(size=31,u0=u0,v0=v0,delta_x=delta,delta_y=delta)
            
            filtered = cv2.filter2D(image, cv2.CV_32F, np.real(gabor_filter))
            
            block_h = image.shape[0] // 4
            block_w = image.shape[1] // 4
            
            for i in range(4):
                for j in range(4):
                    block = filtered[i*block_h:(i+1)*block_h, j*block_w:(j+1)*block_w]
                    energy = np.mean(np.abs(block))
                    features.append(energy)
    
    return np.array(features)


In [23]:
def extract_all_features(image):    
    features = {}

    lbp_hist = lbp_feature(image)
    features.update(lbp_hist) 
    
    texture_features = texture_feature(image)
    features.update(texture_features) 

    hu_moments_features = hu_moments(image)
    features.update(hu_moments_features)

    zernike_moment_features = zernike_feature(image)
    features.update(zernike_moment_features)
    
    # color_structure_features = color_structure_descriptor(image)
    # features.update(color_structure_features)
    
    color_features = enhanced_color_feature(image)
    features.update(color_features) 
    
    edge_histogram_features = extract_edge_histogram_features(image)
    features.update(edge_histogram_features)
    
    # color_features = color_feature(image)
    # features.update(color_features) 

    shape_features = basic_feature(image)
    if shape_features is not None:
        features.update(shape_features)

    glcm_features = compute_glcm_descriptor(image)
    if glcm_features is not None:
        for i, val in enumerate(glcm_features):
            features[f'GLCM_{i}'] = val

    # gist_features = compute_gist_descriptor(image)
    # if gist_features is not None:
    #     for i, val in enumerate(gist_features):
    #         features[f'GIST_{i}'] = val

    return features


def process_directory(base_path):
    all_data = []
    image_paths = []

    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                label = 0 if 'Negative' in root else 1
                image_paths.append((os.path.join(root, file), label))

    for image_path, label in tqdm(image_paths, desc="Processing Images"):
        image = cv2.imread(image_path)
        if image is None:
            continue  

        features = extract_all_features(image)
        features["Label"] = label
 
        all_data.append(features)

    df = pd.DataFrame(all_data)

    return df

In [24]:
base_path = r'/home/duyle/Rice_photos/BC-15'
df = process_directory(base_path)

Processing Images:   0%|          | 0/3677 [00:00<?, ?it/s]/home/duyle/.local/lib/python3.13/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
Processing Images:   9%|▉         | 334/3677 [00:13<02:18, 24.19it/s]


KeyboardInterrupt: 

In [20]:
df.to_csv(f'all_with_zernike_pluscolor_enahnced_edgeenhanced_BC-15.csv',index=False)

In [38]:
types = ['BC-15','Huongthom','Nep87','Q5','Thien_uu','Xi23']
for type in types:
    df = process_directory(f'/home/duyle/Rice_photos/{type}')
    df.to_csv(f'all_with_zernike_pluscolor_enahnced_edge_{type}.csv',index=False)

Processing Images:   0%|          | 0/3677 [00:00<?, ?it/s]/home/duyle/.local/lib/python3.13/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
Processing Images: 100%|██████████| 4145/4145 [05:11<00:00, 13.31it/s]


In [17]:
df

,texture_mean,texture_std,texture_uniformity,texture_third_moment,zernike_0,zernike_1,zernike_2,zernike_3,zernike_4,zernike_5,...,blue_sqr,area,length,width,ratio,major_axis_length,minor_axis_length,convex_hull_area,convex_hull_perimeter,Label
0,152.819736,20.038248,0.039437,3596.159707,0.31831,1.502534e-04,0.462525,0.196733,0.013276,0.014046,...,9.844644,12715,234,76,3.078947,222.841080,73.246323,12846.0,504.984424,0
1,145.484199,25.422890,0.040703,7659.362885,0.31831,5.507275e-04,0.470892,0.193626,0.025171,0.012693,...,9.724670,10462,212,71,2.985915,200.241364,66.632545,10546.5,457.492825,0
2,145.106173,23.235121,0.037482,6440.521765,0.31831,1.746873e-15,0.456747,0.193269,0.017059,0.010133,...,9.765762,10910,208,74,2.810811,196.175446,70.593536,10960.0,454.703499,0
3,153.339070,21.440033,0.041950,4652.943119,0.31831,6.199011e-04,0.463229,0.187735,0.011793,0.012145,...,9.852617,11398,211,75,2.813333,200.679810,72.684647,11476.0,467.173310,0
4,140.540285,30.972378,0.030548,7522.928417,0.31831,3.607720e-04,0.474892,0.187633,0.025607,0.026244,...,9.372739,12380,231,77,3.000000,213.790176,75.759109,12794.5,501.815039,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3672,139.799828,30.436637,0.033865,5615.198825,0.31831,2.206260e-04,0.481884,0.199837,0.026296,0.015629,...,9.477512,11610,240,71,3.380282,227.888321,65.534615,11902.0,512.573571,1
3673,154.848193,17.776393,0.043520,2010.044399,0.31831,1.209034e-04,0.478281,0.187704,0.016870,0.013741,...,10.004896,10575,214,72,2.972222,200.312515,67.399651,10729.5,461.124763,1
3674,138.229146,26.467132,0.037670,9395.690582,0.31831,6.532614e-04,0.462514,0.201591,0.005101,0.005455,...,9.252532,10770,220,69,3.188406,209.212097,65.828743,10929.0,472.594579,1
3675,145.538925,23.635546,0.039149,2987.048113,0.31831,2.047679e-04,0.491030,0.201622,0.027571,0.013138,...,9.576948,9388,228,62,3.677419,214.034332,55.337917,9476.5,477.018385,1
